In [7]:
import pdfplumber
import csv

# Define paths
pdf_path = "/Users/subhadramishra/Desktop/Luddy-Hackathon-2025/New_repo/Fabulous-repo/rmp-csci.pdf"
output_csv_path = "/Users/subhadramishra/Desktop/Luddy-Hackathon-2025/New_repo/Fabulous-repo/professors_data.csv"

# Initialize an empty list to store professor data
professors_data = []

# Function to parse professor details from extracted text
def parse_professor_details(lines):
    professor = {}
    for i, line in enumerate(lines):
        # Extract name and department
        if "Computer Science" in line:
            professor["Department"] = "Computer Science"
            professor["School"] = "Indiana University Bloomington"
            if i > 0:  # Name is typically above "Computer Science"
                professor["Name"] = lines[i - 1].strip()
        # Extract quality rating
        if "QUALITY" in line:
            professor["Quality"] = lines[i + 1].strip() if i + 1 < len(lines) else "N/A"
        # Extract number of ratings
        if "ratings" in line:
            professor["Number of Ratings"] = line.split(" ")[0].strip()
        # Extract "would take again" percentage
        if "%" in line and "would take again" in line:
            professor["Would Take Again"] = line.split("%")[0].strip() + "%"
        # Extract difficulty level
        if "level of difficulty" in line:
            professor["Difficulty"] = line.split(" ")[0].strip()
        
        # Append professor data if complete
        if len(professor) == 7:  # Ensure all fields are populated
            professors_data.append(professor)
            professor = {}  # Reset for next entry

# Extract text from the PDF
try:
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            text = page.extract_text()
            if text:  # Skip empty pages
                lines = text.split("\n")
                parse_professor_details(lines)
except FileNotFoundError:
    print(f"The PDF file does not exist at {pdf_path}. Please check the path.")
except Exception as e:
    print(f"An error occurred while processing the PDF: {e}")

# Write data to CSV file
try:
    with open(output_csv_path, mode="w", newline="", encoding="utf-8") as csv_file:
        fieldnames = ['Name', 'Department', 'School', 'Quality', 'Number of Ratings', 'Would Take Again', 'Difficulty']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(professors_data)
    print(f"Data successfully written to {output_csv_path}")
except Exception as e:
    print(f"An error occurred while writing the CSV file: {e}")

# Print number of professors extracted for verification
print(f"Number of professors extracted: {len(professors_data)}")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Data successfully written to /Users/subhadramishra/Desktop/Luddy-Hackathon-2025/New_repo/Fabulous-repo/professors_data.csv
Number of professors extracted: 70


In [16]:
import pdfplumber
import csv
import re

# Define paths
pdf_path = "/Users/subhadramishra/Desktop/Luddy-Hackathon-2025/New_repo/Fabulous-repo/rmp-informatics.pdf"
output_csv_path = "/Users/subhadramishra/Desktop/Luddy-Hackathon-2025/New_repo/Fabulous-repo/informatics_professors_data.csv"

# Extract all text from PDF first
all_text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            all_text += text + "\n"

# Process extracted text
professors_data = []

# Find all professor blocks and process them
blocks = re.split(r'QUALITY', all_text)[1:]  # Split by QUALITY and remove first empty part

for i, block in enumerate(blocks):
    # Skip blocks that are clearly advertisements or navigation elements
    if any(x in block for x in ["Advertisement", "FROM THE WEB", "https://www.ratemyprofessors.com/search"]):
        continue
        
    lines = block.strip().split('\n')
    
    # Skip if too few lines
    if len(lines) < 2:
        continue
    
    professor = {
        'Department': 'Informatics',
        'School': 'Indiana University Bloomington'
    }
    
    # Extract name (first non-empty line)
    for line in lines:
        if line.strip() and not line.strip().startswith("4/5/25") and not line.strip() == "Search professors at Indiana University Bloomington | Rate My Professors":
            professor['Name'] = line.strip()
            break
    
    # Extract Quality (looking for numbers before or after Informatics)
    for i, line in enumerate(lines):
        # Pattern 1: Number directly before "Informatics"
        quality_match = re.search(r'(\d+\.\d+)\s+Informatics', line)
        # Pattern 2: Number on its own line (like in the image)
        if not quality_match and i > 0 and re.match(r'^\s*\d+\.\d+\s*$', lines[i-1]):
            professor['Quality'] = lines[i-1].strip()
            continue
        if quality_match:
            professor['Quality'] = quality_match.group(1)
            break
    
    # Extract number of ratings
    ratings_match = re.search(r'(\d+)\s+ratings', block)
    if ratings_match:
        professor['Number of Ratings'] = ratings_match.group(1)
    
    # Extract would take again
    take_again_match = re.search(r'(\d+)%\s+would take again', block)
    if take_again_match:
        professor['Would Take Again'] = f"{take_again_match.group(1)}%"
    
    # Extract difficulty
    difficulty_match = re.search(r'(\d+(?:\.\d+)?)\s+level of difficulty', block)
    if difficulty_match:
        professor['Difficulty'] = difficulty_match.group(1)
    
    # Check if we have all required fields
    required_fields = ['Name', 'Department', 'School', 'Quality', 'Number of Ratings', 'Would Take Again', 'Difficulty']
    if all(field in professor for field in required_fields):
        professors_data.append(professor)

# Write data to CSV file
with open(output_csv_path, 'w', newline='', encoding='utf-8') as f:
    fieldnames = ['Name', 'Department', 'School', 'Quality', 'Number of Ratings', 'Would Take Again', 'Difficulty']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(professors_data)

print(f"Successfully extracted {len(professors_data)} professors to {output_csv_path}")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Successfully extracted 29 professors to /Users/subhadramishra/Desktop/Luddy-Hackathon-2025/New_repo/Fabulous-repo/informatics_professors_data.csv
